# <b>双均线策略</b>

版本 v1.0

## <b>目录</b>
* ### <b>双均线策略的交易规则</b>

* ### <b>策略构建步骤</b>

* ### <b>策略的实现</b>

## <b>正文</b>

### <b>一、双均线策略的交易规则</b>

* 当收盘价5日均线大于10日均线时，以第二日开盘价买入；
* 买入后，当收盘价的5日均线小于10日均线时，以第二日开盘价卖出；

### <b>二、策略构建步骤</b>
### 1、确定股票池和回测时间
* 通过证券代码列表输入要回测的单只/多只股票，以及回测的起止日期

### 2、确定买卖条件信号
* 在输入特征列表中通过表达式引擎定义 buy_condition=where(mean(close_0,5)>mean(close_0,10),1,0)，实现买入信号。
* 在输入特征列表中通过表达式引擎定义 sell_condition=where(mean(close_0,5)<mean(close_0,10),1,0)，实现卖出信号。
* 通过基础特征和衍生特征抽取模块实现买卖条件指标 buy_condition 和 sell_condition 数据的抽取。
* 通过缺失数据处理模块删去有缺失值的数据。

### 3、确定买卖原则
* 已有持仓中满足卖出条件的股票为卖出股票列表，需执行卖出操作
* 满足买入条件且没有持仓的股票为买入股票列表，需执行买入操作
* 满足买入条件且已有持仓的股票为调仓股票列表，需执行调整仓位操作
* 本策略中将买入股票列表和调仓股票列表中的所有股票统一调整为等资金比例仓位。

### 4、模拟回测
* 通过 trade 模块中的初始化函数定义交易手续费和滑点；
* 通过 trade 模块中的准备函数定义 context.daily_stock_buy 和 context.daily_stock_sell 变量来获取并存放每日买卖交易信号；
* 通过 trade 模块中的主函数(handle函数)查看每日的买卖交易信号，按照买卖原则执行相应的买入/卖出/调仓操作。

### <b>三、策略的实现</b>
可视化策略实现如下：

<img src='https://bigquant.com/community/uploads/default/original/3X/4/0/409829c44d651407554a43a78c1c2deaabc22f74.jpeg' width=1200 height=1200>

In [3]:
# 本代码由可视化策略环境自动生成 2022年3月25日 14:38
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')  
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 记录用于买入股票的可用现金,因为是早盘卖股票，需要记录卖出的股票市值并在买入下单前更新可用现金；
    # 如果是早盘买尾盘卖，则卖出时不需更新可用现金，因为尾盘卖出股票所得现金无法使用
    cash_for_buy = context.portfolio.cash    
    
    try:
        buy_stock = context.daily_stock_buy[today]  # 当日符合买入条件的股票
    except:
        buy_stock=[]  # 如果没有符合条件的股票，就设置为空
    
    try:
        sell_stock = context.daily_stock_sell[today]  # 当日符合卖出条件的股票
    except:
        sell_stock=[] # 如果没有符合条件的股票，就设置为空
    
    # 需要卖出的股票:已有持仓中符合卖出条件的股票
    stock_to_sell = [ i for i in stock_hold_now if i in sell_stock ]
    # 需要买入的股票:没有持仓且符合买入条件的股票
    stock_to_buy = [ i for i in buy_stock if i not in stock_hold_now ]  
    # 需要调仓的股票：已有持仓且不符合卖出条件的股票
    stock_to_adjust=[ i for i in stock_hold_now if i not in sell_stock ]
    
    # 如果有卖出信号
    if len(stock_to_sell)>0:
        for instrument in stock_to_sell:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            cur_position = context.portfolio.positions[sid].amount # 持仓
            if cur_position > 0 and data.can_trade(sid):
                context.order_target_percent(sid, 0) # 全部卖出 
                # 因为设置的是早盘卖出早盘买入，需要根据卖出的股票更新可用现金；如果设置尾盘卖出早盘买入，则不需更新可用现金(可以删除下面的语句)
                cash_for_buy += stock_hold_now[instrument]
    
    # 如果有买入信号/有持仓
    if len(stock_to_buy)+len(stock_to_adjust)>0:
        weight = 1/(len(stock_to_buy)+len(stock_to_adjust)) # 每只股票的比重为等资金比例持有
        for instrument in stock_to_buy+stock_to_adjust:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            if  data.can_trade(sid):
                context.order_target_value(sid, weight*cash_for_buy) # 买入
# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()

    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)

    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)

    # 每日买入股票的数据框
    context.daily_stock_buy= df.groupby('date').apply(open_pos_con)
    # 每日卖出股票的数据框
    context.daily_stock_sell= df.groupby('date').apply(close_pos_con)

m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
buy_condition=where(mean(close_0,5)>mean(close_0,10),1,0)
sell_condition=where(mean(close_0,5)<mean(close_0,10),1,0)""",
    m_cached=False
)

m2 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2019-03-01'),
    end_date=T.live_run_param('trading_date', '2021-06-01'),
    market='CN_STOCK_A',
    instrument_list="""600519.SHA
600333.SHA""",
    max_count=0
)

m7 = M.general_feature_extractor.v7(
    instruments=m2.data,
    features=m1.data,
    start_date='',
    end_date='',
    before_start_days=60
)

m8 = M.derived_feature_extractor.v3(
    input_data=m7.data,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m4 = M.dropnan.v2(
    input_data=m8.data
)

m3 = M.trade.v4(
    instruments=m2.data,
    options_data=m4.data,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.HIX'
)


[2022-03-25 14:36:22.919783] INFO: moduleinvoker: input_features.v1 开始运行..

[2022-03-25 14:36:22.922645] ERROR: moduleinvoker: module name: input_features, module version: v1, trackeback: PermissionError: [Errno 13] Permission denied: '/var/app/data/bigquant/datasource/user/intermediate/1'\n

PermissionError: [Errno 13] Permission denied: '/var/app/data/bigquant/datasource/user/intermediate/1'